In [38]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.tree import DecisionTreeRegressor


In [39]:
crime = pd.read_csv('/Users/Allison/desktop/sfdat22/notebooks/train.csv')

In [40]:
crime["Dates"] = pd.to_datetime(crime["Dates"])
crime.set_index("Dates", inplace=True)

In [41]:
crime['Year'] = crime.index.year
crime['Month'] = crime.index.month
crime['Day'] = crime.index.dayofweek
crime['Hour'] = crime.index.hour
crime.head()

,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Year,Month,Day,Hour
Dates,,,,,,,,,,,,
2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,2,23
2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,2,23
2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,2,23
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,2,23
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,2,23


In [ ]:
# crime.dtypes[crime.dtypes.map(lambda x: x == 'object')]


In [42]:
# crime = crime.drop('Dates',1)
crime = crime.drop('Descript',1)
crime = crime.drop('Resolution',1)
crime = crime.drop('Address',1)
crime = crime.drop('DayOfWeek',1)
crime.head()

,Category,PdDistrict,X,Y,Year,Month,Day,Hour
Dates,,,,,,,,
2015-05-13 23:53:00,WARRANTS,NORTHERN,-122.425892,37.774599,2015,5,2,23
2015-05-13 23:53:00,OTHER OFFENSES,NORTHERN,-122.425892,37.774599,2015,5,2,23
2015-05-13 23:33:00,OTHER OFFENSES,NORTHERN,-122.424363,37.800414,2015,5,2,23
2015-05-13 23:30:00,LARCENY/THEFT,NORTHERN,-122.426995,37.800873,2015,5,2,23
2015-05-13 23:30:00,LARCENY/THEFT,PARK,-122.438738,37.771541,2015,5,2,23


In [43]:
crime['PdDistrict'] = pd.factorize(crime.PdDistrict)[0]

In [44]:
crime['Category'] = pd.factorize(crime.Category)[0]

In [48]:
feature_cols=['PdDistrict','X','Y','Year','Month','Day','Hour']
crime.head()

,Category,PdDistrict,X,Y,Year,Month,Day,Hour
Dates,,,,,,,,
2015-05-13 23:53:00,0,0,-122.425892,37.774599,2015,5,2,23
2015-05-13 23:53:00,1,0,-122.425892,37.774599,2015,5,2,23
2015-05-13 23:33:00,1,0,-122.424363,37.800414,2015,5,2,23
2015-05-13 23:30:00,2,0,-122.426995,37.800873,2015,5,2,23
2015-05-13 23:30:00,2,1,-122.438738,37.771541,2015,5,2,23


In [49]:
from sklearn.ensemble import RandomForestRegressor
rfreg = RandomForestRegressor()

In [50]:
# define X and y
X = crime[feature_cols]
y = crime.Category

In [ ]:
# list of values to try for max_depth
max_depth_range = range(1, 21)

# list to store the average RMSE for each value of max_depth
RMSE_scores = []

# use 10-fold cross-validation with each value of max_depth
from sklearn.cross_validation import cross_val_score
for depth in max_depth_range:
    treereg = DecisionTreeRegressor(max_depth=depth, random_state=1)
    MSE_scores = cross_val_score(treereg, X, y, cv=10, scoring='mean_squared_error')
    RMSE_scores.append(np.mean(np.sqrt(-MSE_scores)))

In [ ]:
# plot max_depth (x-axis) versus RMSE (y-axis)
plt.plot(max_depth_range, RMSE_scores)
plt.xlabel('max_depth')
plt.ylabel('RMSE (lower is better)')

In [ ]:
# show the best RMSE and the corresponding max_depth
sorted(zip(RMSE_scores, max_depth_range))[0]

(5.9700741241167021, 10)

In [ ]:
# max_depth=10 was best, so fit a tree using that parameter
treereg = DecisionTreeRegressor(max_depth=10, random_state=1)
treereg.fit(X, y)